In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display
from datetime import date, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'

import ptm_lib as ptm
%load_ext autoreload
%autoreload 2

# import pandas_datareader as pdr
# from pandas_datareader import wb
# import quandl
# import matplotlib.pyplot as plt
# from matplotlib import colors

In [ ]:
from_date = date(1928, 1, 1) 
today = date.today()
file = 'MRKT_INDXS'

# S&P500:     Tracks the stock performance of 500 large companies listed on stock exchanges in the United States.
# Nasdaq 100: Tracks the performance of the 100 largest non-financial companies listed on the Nasdaq Stock Market.
# Nasdaq Composite: Tracks all stocks listed on the Nasdaq Stock Market.
# DJIA (Dow Jones): Consists of 30 large, publicly traded companies based in the United States, selected to represent various sectors of the economy.
# Eurostoxx 50:  Tracks the performance of the 50 largest companies in the Eurozone, representing various sectors.
# Eurostoxx 600: A broader index representing 600 large, mid, and small-cap stocks from 17 Eurozone countries.
# FTSE 100:   Comprises the 100 largest companies listed on the London Stock Exchange (LSE). It's a key benchmark for the UK stock market.
# FTSE 250:   Tracks the 250 next largest companies after the FTSE 100 on the LSE, representing mid-cap stocks.
# FTSE 350:   Combines the FTSE 100 and FTSE 250, offering a broader representation of the UK stock market.
# DAX 30:     Represents the 30 largest and most actively traded companies listed on the Frankfurt Stock Exchange in Germany.
# HIS (Hang Seng Index): Tracks the performance of the largest and most liquid companies listed on the Hong Kong Stock Exchange.
# HSCEI (Hang Seng China Enterprises Index): Measures the performance of H-shares, which are shares of Chinese companies listed on the Hong Kong Stock Exchange.
# Nikkei 225: Represents the performance of 225 blue-chip companies listed on the Tokyo Stock Exchange in Japan.
# ASX 200:    Tracks the performance of the top 200 companies listed on the Australian Securities Exchange (ASX).

tickers = ['^GSPC','^NDX','^IXIC','^DJI','^STOXX50E','^STOXX','^FTSE','^FTMC','^FTLC','^GDAXI','^HSI','^HSCE','^N225','^AXJO']
descs = ['S&P500','Nasdaq 100','Nasdaq Composite','Dow Jones','Eurostoxx 50', 'Eurostoxx 600', 'FTSE 100', 'FTSE 250','FTSE 350','DAX 30','HIS','HSCEI','Nikkei 225','ASX 200']

try:
    data = pd.read_csv('Tickers/'+file+'.csv', index_col=0, parse_dates=True, dayfirst=True, header=[0, 1])
except:
    data = yf.download(tickers, from_date, today, interval="1d")
    data = data.swaplevel(axis=1)
    data = data.loc[:, pd.IndexSlice[:, ['Adj Close', 'High']]]    
    data = data.astype('float64')
    data.index = data.index.strftime('%d-%m-%Y')
    data = data.loc[:, tickers]
    data.to_csv('Tickers/'+file+'.csv', index=True)
    with pd.ExcelWriter('Tickers/'+file+'.xlsx', engine='xlsxwriter') as writer:
        for tic in tickers:        
            data[tic].dropna().to_excel(writer, sheet_name=tic, index=True)   # Write each dataframe to a separate worksheet.

data = data.loc[:, tickers]
data

In [ ]:

def market_analysis(data, tickers):    
#     for ind, symb in enumerate(tickers):
#     data[(symb,'RollHigh')] = [data.loc[:date, (symb,'High')].max() for date in data.index]
#     data[(symb,'RollBearLevel')] = data[(symb,'RollHigh')]*0.8
#     data[(symb,'BullIndex')] = np.where(data[(symb,'Adj Close')] > data[(symb,'RollBearLevel')], data[(symb,'Adj Close')], np.nan)
#     data[(symb,'BearIndex')] = np.where(data[(symb,'Adj Close')] <= data[(symb,'RollBearLevel')], data[(symb,'Adj Close')], np.nan)    

    def market_analysis_helper(tic1):
        df = data[tic1].copy()
        df = df.dropna()
        df['RollHigh'] = [df.loc[:date, 'High'].max() for date in df.index]
        df['RollBearLevel'] = df['RollHigh']*0.8
        df['BullIndex'] = np.where(df['Adj Close'] > df['RollBearLevel'], df['Adj Close'], np.nan)
        df['BearIndex'] = np.where(df['Adj Close'] <= df['RollBearLevel'], df['Adj Close'], np.nan)

        fig = go.Figure()    
        first_non_nan_index = max([df['BullIndex'].first_valid_index(), df['BearIndex'].first_valid_index()])
        
        fig.add_trace(go.Scatter(x=df.loc[first_non_nan_index:,].index, y=df.loc[first_non_nan_index:,'BullIndex'], mode="lines", line=dict(color='green', width=1.5)))
        fig.add_trace(go.Scatter(x=df.loc[first_non_nan_index:,].index, y=df.loc[first_non_nan_index:,'BearIndex'], mode="lines", line=dict(color='red', width=1.5)))
        fig.update_yaxes(title_text=tic1)
                
        max_value = max([df['BullIndex'].max() , df['BearIndex'].max()])        
        ay_lef=-30
        ay_rig=30
        ax_lef=-70
        ax_rig=30
        if pd.notna(df['BullIndex'][-1]):
            indx_lim = df['BearIndex'].last_valid_index()
            indx = df.loc[indx_lim:,'BullIndex'].idxmax()
            ypos = df.loc[indx,'BullIndex']
            txt = (indx.strftime('%d-%b-%y')+" "+"{:.2f}".format(ypos))
            fig.add_annotation(x=indx, y=ypos, ax=ax_lef, ay=ay_lef, text=txt, showarrow=True, arrowhead=1, arrowcolor='green',
                                font=dict(color='green', size=14))
            indx = df.index[-1]
            ypos = df.loc[indx,'BullIndex']
            txt = ("{:.2f}".format(ypos))
            fig.add_annotation(x=indx, y=ypos, ax=ax_rig, ay=ay_rig, text=txt, showarrow=True, arrowhead=1, arrowcolor='green',
                                font=dict(color='green', size=14))

        else:
            indx_lim = df['BullIndex'].last_valid_index()
            indx = df.loc[indx_lim:,'BearIndex'].idxmin()
            ypos = df.loc[indx,'BearIndex']
            txt = (indx.strftime('%d-%b-%y')+" "+"{:.2f}".format(ypos))
            fig.add_annotation(x=indx, y=ypos, ax=ax_lef, ay=ay_lef, text=txt, showarrow=True, arrowhead=1, arrowcolor='red',
                                font=dict(color='red', size=14))
            indx = df.index[-1]
            ypos = df.loc[indx,'BearIndex']
            txt = ("{:.2f}".format(ypos))
            fig.add_annotation(x=indx, y=ypos, ax=ax_rig, ay=ay_rig, text=txt, showarrow=True, arrowhead=1, arrowcolor='red',
                                font=dict(color='red', size=14))
        
        fig.update_xaxes(rangeslider_visible=True)
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
        fig.update_traces(showlegend=False)
        fig.show()
        
        fig = go.Figure()    
        fig.add_trace(
            go.Table(
                header=dict(
                    values=['Date']+list(df.columns),
                    font=dict(size=10),
                    align="center"
                ),
                cells=dict(
                    values=[list(df.index.strftime('%d-%m-%Y'))]+[df[k].round(3).tolist() for k in df.columns[0:]],
                    align = "left")
            )
        )
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
        fig.show()

    controls = widgets.interactive(market_analysis_helper, tic1=tickers)
    display(controls)

In [ ]:
market_analysis(data, tickers)